In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Pobranie potrzebnych zasobów NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Funkcja do czyszczenia i preprocessing tekstu
def preprocess_text(text):
    # Usunięcie znaków interpunkcyjnych
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Konwersja na małe litery
    text = text.lower()
    
    # Tokenizacja
    tokens = text.split()
    
    # Usunięcie stopwords i lematyzacja
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return ' '.join(tokens)

# Główna funkcja
def main():
    # Wczytanie danych z lokalnego pliku
    try:
        # Plik spam.csv powinien być w tym samym folderze co skrypt
        df = pd.read_csv('spam.csv', encoding='latin-1')
        # Wybieramy tylko potrzebne kolumny i zmieniamy ich nazwy
        df = df[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'text'})
        print("Dane wczytane pomyślnie. Liczba wiadomości:", len(df))
    except Exception as e:
        print("Błąd wczytywania pliku:", e)
        return

    # Przykładowe wiadomości
    print("\nPrzykładowe wiadomości:")
    print(df.head())

    # Rozkład klas
    print("\nRozkład klas:")
    print(df['label'].value_counts())

    # Preprocessing tekstu
    print("\nPrzetwarzanie tekstu...")
    df['cleaned_text'] = df['text'].apply(preprocess_text)

    # Podział danych na zbiór treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(
        df['cleaned_text'], 
        df['label'], 
        test_size=0.3, 
        random_state=42,
        stratify=df['label']  # Zachowujemy proporcje klas
    )

    # Tworzenie modelu: TF-IDF + Naive Bayes
    print("\nTrenowanie modelu...")
    model = make_pipeline(
        TfidfVectorizer(
            max_features=5000,  # Ograniczamy do 5000 najważniejszych słów
            ngram_range=(1, 2)  # Uwzględniamy również bigramy
        ),
        MultinomialNB(alpha=0.1)  # Dodałem parametr wygładzania
    )

    # Trenowanie modelu
    model.fit(X_train, y_train)

    # Predykcja na zbiorze testowym
    y_pred = model.predict(X_test)

    # Ocena modelu
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nAccuracy:", accuracy_score(y_test, y_pred))

    # Przykład użycia modelu do klasyfikacji nowej wiadomości
    test_messages = [
        "Congratulations! You've won a $1000 prize! Click here to claim!",
        "Hi John, just reminding about our meeting tomorrow at 10am",
        "URGENT: Your bank account has been compromised. Click to verify",
        "The report you requested is attached. Best regards, Team"
    ]

    print("\nTestowanie na przykładowych wiadomościach:")
    for msg in test_messages:
        cleaned_msg = preprocess_text(msg)
        prediction = model.predict([cleaned_msg])[0]
        print(f"\nWiadomość: '{msg}'")
        print(f"Klasyfikacja: {prediction}")
        print(f"Prawdopodobieństwo: {model.predict_proba([cleaned_msg]).max():.2f}")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kamil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kamil\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kamil\AppData\Roaming\nltk_data...


Dane wczytane pomyślnie. Liczba wiadomości: 5572

Przykładowe wiadomości:
  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

Rozkład klas:
label
ham     4825
spam     747
Name: count, dtype: int64

Przetwarzanie tekstu...

Trenowanie modelu...

Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1448
        spam       0.98      0.87      0.92       224

    accuracy                           0.98      1672
   macro avg       0.98      0.93      0.96      1672
weighted avg       0.98      0.98      0.98      1672


Confusion Matrix:
[[1444    4]
 [  29  195]]

Accuracy: 0.9802631578947368

Testowanie na przykładow